# MCS enumeration in the ecoli_core_model

This notebook provides an example on how to use *metaconvexpy* to calculate minimal cut sets coupled with the model reading and flux analysis capabilities of the *COBRApy* package.

## Model reading and setup

Download the model in SBML format from the BiGG models repository. In this example we use the central carbon metabolic model of *Escherichia coli*, downloaded and stored in a temporary folder/path stored as *model_path*

In [1]:
import urllib

model_url = "http://bigg.ucsd.edu/static/models/e_coli_core.xml"
model_path, model_content = urllib.request.urlretrieve(model_url)

Read the SBML temporary file generated from the HTTP request content and generate a COBRA Model object from it. Two functions are defined to assist in further analyses:
* *revert_bounds* - restores all reaction bounds to their original values
* *apply_kos(**kos**)* - knockout reactions contained in *kos* (any iterable containing reaction IDs)

In [2]:
import cobra

model = cobra.io.sbml3.read_sbml_model(model_path)
model_bounds = {r.id:(r.lower_bound, r.upper_bound) for r in model.reactions}

def revert_bounds():
    for r in model.reactions:
        r.lower_bound, r.upper_bound = model_bounds[r.id]

def apply_kos(kos):
    for ko in kos:
        model.reactions.get_by_id(ko).knock_out()
        
model

Name,e_coli_core
Memory address,0x07f23d7fd1198
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5 + 1.0*BIOMASS_Ecoli_core_w_GAM
Compartments,"cytosol, extracellular space"


Extracting environmental conditions from the model, assuming the maximum capacity constant $N$ ($N=1000$). This assumed the media is defined when a lower or upper bound is different than 0, $N$ or $-N$.

This will be needed some steps further for the MCSEnumerator approach.

In [3]:
{r.id:(r.lower_bound, r.upper_bound) 
 for r in model.reactions 
 if (r.lower_bound not in [-1000, 0]) or (r.upper_bound not in [0, 1000])}

{'ATPM': (8.39, 1000.0), 'EX_glc__D_e': (-10.0, 1000.0)}

## Import *metaconvexpy* and enumerate minimal cut sets

Import the MCSEnumerator wrapper class (KShortestMCSEnumeratorWrapper) that works with COBRA models. *KShortestMCSEnumeratorWrapper* provides a simple interface to enumerate minimal cut sets (MCSs) using the K-shortest algorithm (commonly referred to as the MCSEnumerator approach).

MCSs are sets of reactions that disable a certain phenotype in a model. This phenotype is defined as a set of flux or yield constraints. In this example, we will enumerate some MCSs.

In [4]:
from src.metaconvexpy.utilities.external_wrappers import KShortestMCSEnumeratorWrapper

Define the region of the flux space that the MCSs are supposed to block. In this scenario, we are attempting to improve succinate production using glucose as a carbon source. The following IDs are required:
* Succinate exchange : 'EX_succ_e'
* Glucose exchange: 'EX_glc__D_e'
* Maintenance ATP: 'ATPM'

As such, we wish to block low production phenotypes by adding a lower bound on the yield between succinate and glucose. Since the latter is only consumed, we assume its flux value is always negative. Thus, we want yields to be as low as possible, hence the lower bound definition.

In [5]:
yield_space = {
   #('product_id'),('substrate_id'):(lower_bound, upper_bound, deviation - set to zero for most cases)
    ('EX_succ_e','EX_glc__D_e'):(-0.001, None, 0), 
}

flux_space = {
   # flux id     : (lower_bound, upper_bound)
    'EX_glc__D_e': (-10, None),
    'ATPM' : (8.39, None)
}

Having defined the flux space, we can now start the algorithm. The wrapper for MCSEnumerator requires the following parameters:
* *model*: A model instance from COBRA, framed or any other framework if a ModelReader subclass is implemented for it and added to the list of readers
* *target_flux_space_dict*: A dictionary (str -> tuple(float, float)) mapping reaction identifiers to flux bounds representing the target flux space for minimal cut sets.
* *target_yield_space_dict*: A dictionary ((str, str) -> tuple(float, float, float)) mapping product/substrate pairs to a lower and upper bound for the yield constraint between both fluxes and a deviation value for the constraint.
* *algorithm_type*: Can be one of two values:
    * Enumerate one size at a time: *KShortestMCSEnumeratorWrapper.ALGORITHM_TYPE_POPULATE*
    * Enumerate one solution at a time: *KShortestMCSEnumeratorWrapper.ALGORITHM_TYPE_ITERATIVE*
* *stop_criteria*: An integer defining the maximum number of iterations. If the POPULATE routine is used, the algorithm will yield all solutions that have size <stop_criteria> and below. On the other hand, if ITERATIVE is used, the algorithm will yield the <stop_criteria>-shortest solutions.

In [6]:
ksefm = KShortestMCSEnumeratorWrapper(
    model=model,
    target_flux_space_dict=flux_space, 
    target_yield_space_dict=yield_space, 
    algorithm_type=KShortestMCSEnumeratorWrapper.ALGORITHM_TYPE_POPULATE, 
    stop_criteria=5)

The MILP problems aren't actually run once the KShortestMCSEnumeratorWrapper instance is created. To obtain the solutions, the .get_enumerator() method must be called. This method returns a generator that can be called in several ways.

In [7]:
enumerator = ksefm.get_enumerator()

Depending on time constraints, one can interactively call the *next* function on the enumerator so it iterates just once. This is useful when the problem is large or when exploring new constraints.

In [8]:
solutions = next(enumerator)

Starting size 1


Solutions are always lists of dictionaries mapping fluxes with values for all active fluxes. For MCSs, one simply has to obtain the dictionary keys for all elements of the list

In [9]:
mcs = [set(d.keys()) for d in solutions]
mcs

[{'ATPM'},
 {'GLCpts'},
 {'EX_glc__D_e'},
 {'GAPD'},
 {'PGK'},
 {'ENO'},
 {'PGM'},
 {'EX_h2o_e'},
 {'GLUSy', 'H2Ot'},
 {'EX_succ_e'},
 {'SUCCt3'},
 {'EX_h_e'},
 {'NADH16'}]

Unlike the previous approach, we can try and extensively enumerate all MCSs. We must redefine the enumerator as it already iterated once. We can import the *chain* function and use it to get a list of solutions until the stop criteria is reached.

In [10]:
enumerator = ksefm.get_enumerator()

In [11]:
from itertools import chain
solutions = list(chain(*ksefm.get_enumerator()))

Starting size 1
Starting size 2
Starting size 3
Starting size 4
Starting size 5


In [12]:
mcs = [set(e.keys()) for e in solutions]
mcs[1:10]

[{'GLCpts'},
 {'EX_glc__D_e'},
 {'GAPD'},
 {'PGK'},
 {'ENO'},
 {'PGM'},
 {'EX_h2o_e'},
 {'GLUSy', 'H2Ot'},
 {'EX_succ_e'}]

## Constraint-based analysis of mutant phenotypes

In this format, we can easily use COBRA to analyse these knockouts. In this snippet, a dictionary is created to store various simulation values obtained after applying the reaction knockouts suggested by the MCSs. These are:
* *product_minimum* and *product_maximum* - Flux range for succinate production
* *max_growth* - Maximum growth rate
* *product_at_max_growth* - Predicted production rate at maximum growth
* *glucose_consumption* - Glucose consumption rate at maximum growth

In [13]:
def analyse(kos):
    apply_kos(kos)
    try:
        sol = model.optimize()

        biomass = sol.fluxes.BIOMASS_Ecoli_core_w_GAM
        product = sol.fluxes.EX_succ_e
        substrate = sol.fluxes.EX_glc__D_e
        fva = cobra.flux_analysis.variability.flux_variability_analysis(model, reaction_list=['EX_succ_e']).loc['EX_succ_e',:]
        
        return {
        'product_maximum': fva['maximum'],
        'product_minimum': fva['minimum'],
        'max_growth': biomass,
        'product_at_max_growth':product,
        'glucose_consumption': substrate
        }
    
    except:
        pass
    finally:
        revert_bounds()
    return 

*pandas* can be used to store these values as a dataframe for easier manipulation and analysis 

In [14]:
import pandas as pd

df = pd.DataFrame({frozenset(ko): analyse(ko) for ko in mcs}).T

cobra/util/solver.py:408 UserWarning: solver status is 'infeasible'


Solutions with null or very low growth, as well as solutions with very low production rates are filtered out. This way, only the best candidates are displayed

In [15]:
null_growth = df.max_growth.isnull()
zero_growth = df.max_growth <= 1e-10
no_product = df.product_at_max_growth <= 1e-5

df[~ (zero_growth | null_growth | no_product)].sort_values(by='max_growth', ascending=False).iloc[1:20,:]

,glucose_consumption,max_growth,product_at_max_growth,product_maximum,product_minimum
"(O2t, PPS, D_LACt2, PYRt2, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, EX_pyr_e, EX_lac__D_e, O2t, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, EX_pyr_e, CYTBD, LDH_D, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(EX_pyr_e, EX_lac__D_e, PPS, EX_o2_e, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(EX_pyr_e, ACALD, PPS, EX_o2_e, D_LACt2)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, CYTBD, D_LACt2, PYRt2, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, EX_lac__D_e, CYTBD, PYRt2, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, O2t, LDH_D, PYRt2, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, O2t, EX_lac__D_e, PYRt2, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
"(ADK1, EX_pyr_e, O2t, D_LACt2, ACALD)",-10,0.110886,9.09864,9.09864,9.09864
